In [ ]:
# Setting Theano as keras backend
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")

In [1]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, SGD, Nadam, RMSprop
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
# Load weights of convolution layers
from keras import backend as K
vgg_model = VGGFace(include_top=False,weights='vggface', input_shape=(92,112,3))
print('CNN Weights Loaded.')

CNN Weights Loaded.


In [3]:
#custom parameters
nb_class = 5
hidden_dim = 512

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

# Finetuning: only the last 3 fully connected layers are trained. The rest are made non-trainable.
layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count = layer_count+1
for l in range(layer_count-3):
    custom_vgg_model.layers[l].trainable=False
    
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 92, 112, 3)        0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 92, 112, 64)       1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 92, 112, 64)       36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 46, 56, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 46, 56, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 46, 56, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 23, 28, 128)       0         
__________

In [4]:
# Loading train, test and validation data
train_data_dir = '/Users/seba/Desktop/Dataset2/train'
validation_data_dir = '/Users/seba/Desktop/Dataset2/validate'
#test_data_dir = '/Users/seba/Desktop/Dataset/test'
nb_train_samples = 5
nb_validation_samples = 5
#nb_test_samples = 8
img_height = 92
img_width = 112 
batch_size = 5

In [ ]:
#image preprocessing 
def preprocess_input(x):
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

In [5]:
# prepare data augmentation configuration
validate_datagen = ImageDataGenerator(
    preprocessing_function=utils.preprocess_input,
    featurewise_center=True,
    featurewise_std_normalization=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_datagen = ImageDataGenerator(
    preprocessing_function=utils.preprocess_input,
    featurewise_center=True, 
    featurewise_std_normalization=True,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=32,
    class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=32,
    class_mode='categorical')

Found 25 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [6]:
# Compile model 
custom_vgg_model.compile(optimizer=SGD(lr=0.00000000001, momentum=0.7, decay=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,patience=0, min_lr=1e-6)
tensorboard_callback=TensorBoard(log_dir='/Users/seba/Desktop/Face Recognition/Logs/log_1/', histogram_freq=0,write_graph=True,write_images=False)


In [7]:
# Finetuning: train last 3 fully connected layers
custom_vgg_model.fit_generator( 
    train_generator,
    steps_per_epoch=nb_train_samples/5,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/5,callbacks=[tensorboard_callback])

Epoch 1/10


/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1/1 [==============================] - 16s - loss: 6.2119 - acc: 0.2000 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 2/10
1/1 [==============================] - 12s - loss: 8.0515 - acc: 0.1600 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 3/10
1/1 [==============================] - 12s - loss: 9.1036 - acc: 0.1200 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 4/10
1/1 [==============================] - 13s - loss: 10.6181 - acc: 0.0800 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 5/10
1/1 [==============================] - 14s - loss: 8.3896 - acc: 0.1600 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 6/10
1/1 [==============================] - 14s - loss: 7.0171 - acc: 0.2800 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 7/10
1/1 [==============================] - 13s - loss: 6.3017 - acc: 0.2000 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 8/10
1/1 [==============================] - 14s - loss: 8.5763 - acc: 0.1200 - val_loss: 10.8442 - val_acc: 0.1200
Epoch 9/10
1/1 [==========================

In [ ]:
# batch_size??
# Validation_steps??

#The number of samples processed for each epoch is batch_size * steps_per_epochs.
#steps_per_epoch = number of times weights are updated per epoch